## Explore Squad v1.1

In [1]:
%ls



explore_squad_v1_1.ipynb


In [27]:
import os, sys
path_project = '~/fontes/exqa-complearning/'

sys.path.append(os.path.abspath('.'))
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath(r'../..'))
print(sys.path)

['/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/miniconda3/envs/relevar-busca/lib/python37.zip', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/lib-dynload', '', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages', '/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages/IPython/extensions', '/home/borela/.ipython', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration/.\\.', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration', '/home/borela/fontes/exqa-complearning/source/exploration/.\\.', '/home/borela/fontes/exqa-complearning/source/exploration'

In [23]:
%cd ..


/home/borela/fontes/exqa-complearning


In [28]:
%pwd

'/home/borela/fontes/exqa-complearning'

In [30]:
from transformers.data.processors.squad import SquadV1Processor # SquadV2Processor

processor = SquadV1Processor()
examples = processor.get_dev_examples("data/dataset/squad/", filename="dev-v1.1.json")
print(len(examples))

100%|██████████| 48/48 [00:02<00:00, 23.14it/s]

10570


In [31]:
def display_example(qid):    
    from pprint import pprint

    idx = qid_to_example_index[qid]
    q = examples[idx].question_text
    c = examples[idx].context_text
    a = [answer['text'] for answer in examples[idx].answers]
    
    print(f'Example {idx} of {len(examples)}\n---------------------')
    print(f"Q: {q}\n")
    print("Context:")
    pprint(c)
    print(f"\nTrue Answers:\n{a}")



In [32]:
    
# generate some maps to help us identify examples of interest
qid_to_example_index = {example.qas_id: i for i, example in enumerate(examples)}
qid_to_has_answer = {example.qas_id: bool(example.answers) for example in examples}
answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if has_answer]
# no_answer_qids = [qas_id for qas_id, has_answer in qid_to_has_answer.items() if not has_answer]


In [33]:
display_example(answer_qids[1300])


Example 1300 of 10570
---------------------
Q: Where was the  Continental Edison Company located?

Context:
('In 1882, Tesla began working for the Continental Edison Company in France, '
 'designing and making improvements to electrical equipment. In June 1884, he '
 'relocated to New York City:57–60 where he was hired by Thomas Edison to work '
 "at his Edison Machine Works on Manhattan's lower east side. Tesla's work for "
 'Edison began with simple electrical engineering and quickly progressed to '
 'solving more difficult problems.')

True Answers:
['France', 'in France', 'France']


In [43]:
answer_qids[1300], type(answer_qids[1300])

('56e0d54a7aa994140058e76a', str)

In [44]:
qid = answer_qids[1300]

In [45]:
idx = qid_to_example_index[qid]
q = examples[idx].question_text
c = examples[idx].context_text
a = [answer['text'] for answer in examples[idx].answers]
q, c


('Where was the  Continental Edison Company located?',
 "In 1882, Tesla began working for the Continental Edison Company in France, designing and making improvements to electrical equipment. In June 1884, he relocated to New York City:57–60 where he was hired by Thomas Edison to work at his Edison Machine Works on Manhattan's lower east side. Tesla's work for Edison began with simple electrical engineering and quickly progressed to solving more difficult problems.")

In [36]:
import copy


In [37]:
from source.fine_tuning.responder_extracao import responder_extracao, json_exemplo  # pylint: disable=wrong-import-position # precisa dos sys.path antes
from source.fine_tuning.modelo_reader import lista_modelos_reader

modelo reader inicializados: pierreguillou/bert-large-cased-squad-v1.1-portuguese Hora: [2022-Jun-26 18:46:28]


In [38]:
json_fine_tuning = copy.deepcopy(json_exemplo)
json_fine_tuning["tamanho_max_resposta"] = 30


In [ ]:
json_fine_tuning

In [46]:
json_fine_tuning['texto_pergunta'] = q
json_fine_tuning['texto_contexto'] = c

In [49]:
json_fine_tuning

{'texto_pergunta': 'Where was the  Continental Edison Company located?',
 'top_k_reader': 3,
 'tamanho_max_resposta': 30,
 'texto_contexto': "In 1882, Tesla began working for the Continental Edison Company in France, designing and making improvements to electrical equipment. In June 1884, he relocated to New York City:57–60 where he was hired by Thomas Edison to work at his Edison Machine Works on Manhattan's lower east side. Tesla's work for Edison began with simple electrical engineering and quickly progressed to solving more difficult problems.",
 'sigla_modelo_reader': 'pierreguillou/bert-large-cased-squad-v1.1-portuguese'}

In [48]:
resposta = responder_extracao(json_fine_tuning)
print(f"resposta {resposta}")


resposta {'lista_sugestao_resposta': [{'texto_resposta': 'New York City:57–60', 'score': 0.9129816889762878, 'lista_referencia': [[164, 183]]}, {'texto_resposta': 'New York City:57–60 where he was hired by Thomas Edison to work at his Edison Machine Works', 'score': 0.0031480903271585703, 'lista_referencia': [[164, 255]]}]}


In [51]:
json_fine_tuning['texto_pergunta']  = "Em 1882, Tesla começou a trabalhar para a Continental Edison Company na França, projetando e fazendo melhorias em equipamentos elétricos. Em junho de 1884, ele se mudou para Nova York: 57-60, onde foi contratado por Thomas Edison para trabalhar em sua Edison Machine Works, no lado leste inferior de Manhattan. O trabalho de Tesla para Edison começou com engenharia elétrica simples e rapidamente progrediu para resolver problemas mais difíceis."
json_fine_tuning['texto_contexto'] = "Onde estava localizada a Continental Edison Company?"

In [50]:
a

['France', 'in France', 'France']

In [52]:
resposta = responder_extracao(json_fine_tuning)
print(f"resposta {resposta}")


resposta {'lista_sugestao_resposta': [{'texto_resposta': '', 'score': 0.22037574648857117, 'lista_referencia': [[0, 0]]}, {'texto_resposta': 'Onde estava localizada a Continental Edison Company?', 'score': 0.02530529536306858, 'lista_referencia': [[0, 51]]}, {'texto_resposta': 'localizada a Continental Edison Company?', 'score': 0.01168345008045435, 'lista_referencia': [[12, 51]]}]}


In [89]:
del f1_score
del exact_match_score


In [98]:
from source.calculation.metric import qa_metric
# exact_match_score, f1_score, metric_max_over_ground_truths


In [97]:
# %load_ext autoreload
# %autoreload 2

In [93]:
import importlib


In [99]:
importlib.reload(qa_metric)

nova importação
def


<module 'source.calculation.metric.qa_metric' from '/home/borela/fontes/exqa-complearning/source/calculation/metric/qa_metric.py'>

In [101]:
qa_metric.f1_score (prediction=resposta['lista_sugestao_resposta'][1]['texto_resposta'],ground_truth=a[0])

0

In [104]:
qa_metric.exact_match_score(prediction=resposta['lista_sugestao_resposta'][1]['texto_resposta'],ground_truth=a[0])

nova importação
def


0

In [55]:
resposta['lista_sugestao_resposta'][1]['texto_resposta']

'Onde estava localizada a Continental Edison Company?'

In [108]:
qa_metric.metric_score_over_ground_truths(metric_fn= qa_metric.exact_match_score, prediction='france',ground_truths=a)

[1, 0, 1]

In [106]:
qa_metric.metric_max_over_ground_truths(metric_fn= qa_metric.f1_score, prediction='france',ground_truths=a)

1.0

## Config ...